# Event-Studies com Tratamento Contínuo

Prof. Daniel de Abreu Pereira Uhr


## Conteúdo

* Introdução
* Parâmetros Causais de Interesse
* Parâmetros do Tipo Estudo de Evento
* Identificação dos Parâmetros de Estudo de Evento
* Aplicação em Python


## Referências

* Brantly Callaway, Andrew Goodman-Bacon, and Pedro H. C. Sant’Anna. 2024. "Event-Studies with a Continuous Treatment." American Economic Review, PAPERS AND PROCEEDINGS, May, 114(5): 1-8.

### Introdução

Nessa aula vamos aprender sobre o artigo "Event-Studies with a Continuous Treatment" de Callaway, Goodman-Bacon e Sant'Anna (2024). O artigo discute a estimação de parâmetros causais em estudos de eventos com tratamento contínuo. Vamos tentar abordar os principais pontos do artigo e aplicar os conceitos em Python.

Um pesquisador pode analisar um desenho de Diferenças em Diferenças (DiD) de dois grupos e dois períodos sem precisar fazer muitas escolhas. Supondo que o tratamento não afete os resultados antes de sua iniciação, o efeito médio do tratamento sobre os tratados é identificado sob uma suposição de tendências paralelas pela diferença entre as mudanças médias dos resultados entre os grupos: um parâmetro, uma suposição de tendências paralelas e um estimador.

No entanto, um desenho DiD mais complexo exige que os pesquisadores façam mais decisões. Este artigo baseia-se em Callaway, Goodman-Bacon e Sant'Anna (2024) e considera as escolhas que um pesquisador enfrenta ao usar métodos DiD para estudar um tratamento que começa em diferentes momentos (escalonado) e afeta as unidades em diferentes graus (intensidade do tratamento). Discutimos maneiras transparentes de agregar o grande conjunto de parâmetros causais que surgem nesse contexto, transmitindo a heterogeneidade por tempo de evento e entre doses, e mostramos como essas escolhas podem simplificar a estimação e a inferência. Em contraste com os parâmetros de especificações lineares comuns com efeitos fixos de duas vias, nossos parâmetros resumidos não sofrem com problemas de ponderação negativa/não transparente.

### Parâmetros Causais de Interesse
Considere um conjunto de dados em painel com $N$ unidades indexadas por $i$, e $T$ períodos de tempo indexados por $t$. Denotamos o período em que a unidade $i$ é tratada pela primeira vez por $G_i \in G = \{2, \ldots, T, \infty\}$, onde $G_i = \infty$ significa que uma unidade não é tratada até $T$ (“nunca tratada”). Como focamos em configurações escalonadas, $G_i$ pode ser interpretado como o “grupo de tempo de tratamento”. Deixe $D_i \in D \subseteq [0, d_H]$, onde $d_H < \infty$, representar a “dose” (ou intensidade) do tratamento que a unidade $i$ recebeu quando foi tratada pela primeira vez. Interpretamos $D_i$ como o “grupo de dose”.

Adotamos o framework de resultado potencial e escrevemos $Y_{i,t}(g, d)$ como o resultado potencial da unidade $i$ no tempo $t$ se essa unidade fosse tratada pela primeira vez no período $g$, com dose $d$. Para unidades que permanecem não tratadas até o último período de tempo dos dados disponíveis, escrevemos $Y_{i,t}(0) = Y_{i,t}(\infty, 0)$. Essa notação define os efeitos médios do tratamento específicos para grupo-tempo-dose:

$$
ATT(g, t, d) = E[Y_t(g, d) - Y_t(0) \mid G = g, D = d].
$$

$ATT(g, t, d)$ é o efeito médio do tratamento no período $t$ de (i) ser tratado no período $g$ e (ii) experimentar a dose $d$ versus dose zero, entre aqueles no grupo de tempo $g$ que receberam a dose $d$. Os parâmetros $ATT(g, t, d)$ descrevem o que algum tratamento realmente alcançou. Eles incorporam muitos tipos de heterogeneidade que os pesquisadores frequentemente desejam relatar e interpretar. Para um dado $g$ e $t$, os efeitos médios em diferentes doses descrevem a “função dose-resposta” do grupo $g$. Para um dado $d$ e $g$, as diferenças nos efeitos causais médios ao longo de $t$ representam a dinâmica do efeito do tratamento. Finalmente, para um dado $d$ e $t$, diferentes efeitos médios por $g$ capturam uma combinação de dinâmica e diferenças no efeito de ser tratado em um determinado momento.

Com tratamentos contínuos, outra classe de parâmetros causais que pode ser de interesse são os parâmetros de resposta causal média, definidos como:

$$
ACR(g, t, d) = \frac{\partial E[Y_t(g, \tilde{d}) \mid G = g]}{\partial \tilde{d}} \Bigg|_{\tilde{d} = d}.
$$

$ACR(g, t, d)$ é a resposta causal média a uma mudança marginal na dose em $d$ para todas as unidades no grupo de tempo $g$. Parâmetros $ACR$ respondem a perguntas causais sobre qual nível de tratamento importa mais ou menos. Esse parâmetro de inclinação também é uma função de $g$, $t$ e $d$, e pode variar nessas dimensões de maneiras significativas.

Em configurações de DiD escalonadas com tratamentos contínuos, pode não ser prático estimar uma função do tipo dose-resposta para cada $t$ e $g$. Assim, os pesquisadores podem querer agregar esses “blocos de construção” por tempo desde o tratamento (tempo de evento; $e = t - g$) e/ou entre doses. Aqui, focamos em resumir os $ATT(g, t, d)$ ao longo de ambas as margens e discutimos como essas escolhas de agregação moldam a estimação.

### Parâmetros do Tipo Estudo de Evento

Começamos discutindo as agregações de estudo de evento que fazem a média sobre as dosagens de tratamento. Para um dado grupo $g$ e tempo $t$, seja $ATT_o(g, t) = E[ATT(g, t, D) \mid G = g, D > 0]$, que é o efeito médio do tratamento para aquele grupo em um determinado ponto no tempo. Em seguida, definimos:

$$
ATT_{es}(e) = E[ATT_o(G, G + e) \mid G + e \in [2, T], D > 0]
$$

que denota o efeito médio do tratamento entre aqueles que foram expostos a qualquer tratamento por exatamente $e$ períodos, condicional a serem observados participando do tratamento por aquele número de períodos ($G + e \in [2, T]$) e sendo tratados em algum momento ($D > 0$). Quando $D$ é binário, $ATT_{es}(e)$ se reduz ao coeficiente do estudo de evento considerado por Callaway e Sant'Anna (2021).

Os parâmetros acima agregam completamente as doses e, portanto, não descrevem a dinâmica dos efeitos do tratamento para grupos de “alta dose” ou “baixa dose”, por exemplo. Para abordar essas questões, podemos agregar parcialmente as doses em cada grupo e período de tempo para formar os parâmetros de estudo de evento “conscientes da dose”:

$$
ATT_{es_{d1,d2}}(e) = E[ATT_o(G, G + e) \mid d1 \leq D \leq d2, G + e \in [2, T]],
$$

onde $ATT_o(g, g + e) = E[ATT(g, t, D) \mid G = g, d1 \leq D \leq d2]$, e $0 < d1 \leq d2$ são limites dentro do suporte de $D$. Escolhendo diferentes intervalos, podemos avaliar como as dinâmicas do efeito médio do tratamento variam entre grupos de dosagem. Por exemplo, poderíamos definir $d1$ e $d2$ para dividir as unidades tratadas em aquelas com doses acima ou abaixo da mediana. Naturalmente, pode-se considerar partições de dose mais refinadas, mas recomendamos prestar atenção ao tamanho efetivo da amostra em cada partição escolhida.

Uma característica do $ATT_{es_{d1,d2}}(e)$ é que ele agrega entre várias dosagens de tratamento. Em algumas aplicações, no entanto, os pesquisadores podem querer relatar heterogeneidade detalhada com respeito a $d$. Agregar sobre alguns tempos de evento pode facilitar o relato das funções estimadas de dose-resposta. Para formalizar essa ideia, sejam $e1$ e $e2$ dois tempos de evento pós-tratamento tais que $0 \leq e1 \leq e2$ e

$$
ATT_{es_{e1,e2}}(d) = \frac{1}{e2 - e1 + 1} \sum_{e=e1}^{e2} E[ATT(G, G + e, d) \mid G + e2 \in [2, T], D = d]
$$

seja o efeito médio do tratamento de receber a dose $d$, entre todas as unidades que foram tratadas por pelo menos $e2$ períodos, calculado como a média ao longo dos tempos de evento $e1$ até $e2$. Quando $e2 = e1 = e$, $ATT_{es_{e1,e2}}(d)$ fornece uma função dose-resposta para todas as unidades que foram tratadas por exatamente $e$ períodos. Quando $e2 > e1$, no entanto, $ATT_{es_{e1,e2}}(d)$ faz a média dessas curvas ao longo do tempo de evento e pode ser usado para resumir respostas de dose em termos de efeitos de “curto prazo” (ex.: $e1 = 0, e2 = 2$) e de “longo prazo” (ex.: $e1 = 3, e2 = 4$). Ao construir essas curvas de resposta à dose com média sobre os tempos de evento, impomos que os dados estejam balanceados no tempo de evento dentro da janela de $e1$ a $e2$ ($G + e2 \in [2, T]$), para que mudanças de composição não sejam uma preocupação, mas é possível relaxar essa restrição.

A seguir, mostramos que essas funcionais de estudo de evento são identificadas sob suposições tradicionais de DiD.


### Identificação dos Parâmetros de Estudo de Evento

As seguintes suposições são suficientes para identificar os parâmetros de estudo de evento. Seja $D_{+} = D \setminus \{0\}$ o subconjunto positivo do suporte de $D$. Seja $\Delta Y_t = Y_t - Y_{t-1}$ e defina $W_t = D1\{t \geq G\}$.

* **SUPOSIÇÃO 1:** Os dados observados consistem de $\{Y_{i,1}, \ldots, Y_{i,T}, D_i, G_i\}_{i=1}^{n}$, que são independentes e identicamente distribuídos.

* **SUPOSIÇÃO 2:** (i) $D^+ = [d_L, d_U]$ com $0 < d_L < d_U < \infty$, (ii) $P(D = 0) > 0$ e $dF_{D|G}(d | g) > 0$ para todos $(g, d) \in (G \setminus \{\infty\}) \times D^+$, (iii) Para todo $g \in (G \setminus \{\infty\})$ e $t = 2, \ldots, T$, $E[\Delta Y_t | G = g, D = d]$ é continuamente diferenciável em $d$ sobre $D^+$.

* **SUPOSIÇÃO 3:** (i) Para todo $g \in G$ e $t = 1, \ldots, T$ com $t < g$, $Y_{i,t}(g, d) = Y_{i,t}(0)$ a.s., (ii) $W_{i,1} = 0$ a.s. e para $t = 2, \ldots, T$, $W_{i,t-1} = d$ implica que $W_{i,t} = d$ a.s.

* **SUPOSIÇÃO 4:** Para todos $(g, g') \in G \times G$, $t = 2, \ldots, T$, e $(d, d') \in D \times D$, $E[\Delta Y_t(0) | G = g, D = d] = E[\Delta Y_t(0) | G = g', D = d']$.

A Suposição 1 afirma que temos dados de painel. A Suposição 2 afirma que temos um conjunto de unidades que nunca são tratadas e que o tratamento é contínuo; se não houver unidades nunca tratadas, podemos restringir a análise aos períodos $t = 1, \ldots, \bar{G} - 1$, onde $\bar{G} = \max\{G_i : G_i < \infty\}$ é o tempo do último grupo tratado. A Suposição 3 impõe a não-antecipação e que o tratamento é escalonado. Finalmente, a Suposição 4 é uma condição de tendências paralelas que afirma que, na ausência de tratamento, a evolução média dos desfechos potenciais não tratados é a mesma entre grupos de tempo-dosagem. Essas suposições são semelhantes às de Callaway, Goodman-Bacon e Sant'Anna (2024).

Seja $U_t$ uma variável binária genérica que assume o valor um se uma unidade fizer parte de um grupo de comparação “limpo” escolhido pelo usuário no tempo $t$; por exemplo, o indicador de não-tratamento $U_t = 1\{G > t \cup D = 0\}$. Definimos $\theta_o(g, t) = E[Y_t - Y_{g-1} | G = g, d_1 \leq D \leq d_2] - E[Y_t - Y_{g-1} | U_{\max\{t, g-1\}} = 1, d_1 \leq D \leq d_2 \cup D = 0]$, e $\theta_o(g, t) = \theta_{d_L, d_H}(g, t)$.

Além disso, para cada unidade $i$ tal que $G_i + e_2 \in [2, T]$, seja $\tilde{Y}_i(g) = 1\{G_i = g\} \sum_{e=e_1}^{e_2} (Y_{i, g+e} - Y_{i, g-1}) / (e_2 - e_1 + 1) - E[Y_{g+e} - Y_{g-1} | U_{g+e_2} = 1] / (e_2 - e_1 + 1)$.

* **TEOREMA 1:** Sob as Suposições 1 a 4, (i) $ATT_{es}(e) = E[\theta_o(G, G + e) | G + e \in [2, T], D > 0]$, (ii) $ATT_{es_{d1,d2}}(e) = E[\theta_o(G, G + e) | G + e \in [2, T], d_1 \leq D \leq d_2]$, e (iii) $ATT_{es_{e1,e2}}(d) = E[\tilde{Y}_{e1,e2}(G) | G + e_2 \in [2, T], D = d]$.

O Teorema 1(i) mostra que podemos essencialmente ignorar a intensidade do tratamento ao focar nos parâmetros do tipo estudo de evento $ATT_{es}(e)$ e, portanto, usar os estimadores de configurações DiD escalonadas com um tratamento binário para estimar esses parâmetros. Por exemplo, com $U_t = 1\{G > t \cup D = 0\}$, pode-se usar os estimadores de estudo de evento de Callaway e Sant'Anna (2021) com as unidades não tratadas como grupo de comparação; isso envolve “rotular” unidades com $D = 0$ como $G = \infty$.

O Teorema 1(ii) mostra que isso ainda é o caso quando se deseja apresentar estudos de evento que apenas agregam parcialmente entre dosagens. Mais especificamente, para estimar $ATT_{es_{d1,d2}}(e)$, é necessário primeiro submeter as observações apenas às unidades que receberam dose de tratamento entre $d_1$ e $d_2$, ou aquelas que receberam dose zero, $D = 0$. Dentro desse subconjunto de unidades, pode-se então “ignorar” a intensidade do tratamento e proceder com a estimação de $ATT_{es}(e)$.

O Teorema 1(iii) tem uma abordagem ligeiramente diferente das partes anteriores, pois $ATT_{es_{e1,e2}}(d)$ é um parâmetro de resposta à dose. No entanto, também possui implicações práticas importantes. Mais especificamente, destaca que se pode confiar nos estimadores de curva de resposta à dose propostos por Callaway, Goodman-Bacon e Sant'Anna (2024) para configurações de dois períodos, pois $Y_{e1,e2}(g)$ essencialmente envolve apenas um período pré e pós-tratamento. Em configurações com um grande número de unidades transversais, pode-se utilizar o estimador não paramétrico orientado por dados de Chen, Christensen e Kankanala (2023) discutido por Callaway, Goodman-Bacon e Sant'Anna (2024). Quando o tamanho da amostra é limitado, pode-se escolher um modelo paramétrico flexível (por exemplo, splines com um número fixo de nós) para aproximar a curva de resposta à dose.


### Aplicação em Python

Bartik et al. (2019a) usam uma variável de tratamento escalonada e não binária para estudar os efeitos econômicos locais da fraturação hidráulica (fracking). O fracking é possível apenas em áreas com formações de xisto subterrâneas que podem ser fraturadas para liberar hidrocarbonetos. Os custos e rendimentos do fracking—“prospectividade”—dependem de fatores geológicos que variam continuamente entre áreas dentro de uma determinada formação de xisto. Modificamos ligeiramente o desenho de pesquisa DiD em Bartik et al. (2019a) explorando a variação no momento da atividade de fracking em diferentes formações de xisto de 2001 a 2014 ($G_i$, coletados manualmente pelos autores) e a variação contínua na pontuação de prospectividade entre condados ($D_i$, adquirida da Rystad Energy); ver Bartik et al. (2019b). Denominamos os condados com pontuação de prospectividade zero como “nunca tratados” e definimos $G_i = \infty$ para eles. Focamos no logaritmo do emprego total do condado como nosso principal desfecho, e usamos as unidades ainda não tratadas como grupo de comparação em todas as estimativas abaixo.


<div style="text-align:center;">
    <img src="images\CDD1.png"  alt="Imagem" style="width: 700px;"/>
</div>

A Figura 1 apresenta os resultados para $ATT_{es}(e)$ usando dois conjuntos de $(d_1, d_2)$: a curva laranja define $d_1 = 0.20$ e $d_2 = 3.95$, onde 0.20 e 3.95 são, respectivamente, a exposição mínima e mediana ao fracking entre os condados com exposição positiva. Chamamos esse grupo de “baixa dose”. A curva azul define $d_1$ ligeiramente acima de 3.95 e $d_2 = 9.35$, onde o último é a exposição máxima ao fracking. Chamamos esse grupo de “alta dose”. As tendências pré-tratamento parecem paralelas para ambos os grupos por 11 anos antes do fracking, apoiando a suposição de tendências paralelas. Os condados de “baixa dose” têm efeitos de tratamento que não são estatisticamente diferentes de zero até três anos após o início do fracking, quando eles são cerca de 2%. Os condados de “alta dose”, por outro lado, apresentam efeitos médios maiores que crescem de 2% no ano após o fracking para 7% de emprego mais alto quatro anos depois.

A Figura 2 apresenta os resultados para as curvas de resposta à dose com médias ao longo do tempo, $ATT_{es}(d)$: a curva laranja define $e_1$ e $e_2$ para 0 e 2 (“curto prazo”) e a curva azul usa 3 e 4 (“longo prazo”). Usamos splines cúbicos com nós internos nos percentis 25, 50 e 75 da dose. Esta figura ecoa as conclusões da Figura 1 de que os efeitos de curto prazo são menores do que os de longo prazo e que os condados com pontuações de prospectividade mais altas têm maiores efeitos no emprego provenientes do fracking. Essa agregação, no entanto, mostra onde na distribuição da prospectividade esses efeitos são maiores. Por exemplo, no longo prazo, os efeitos médios no emprego são igualmente grandes para todos os condados com pontuações acima de cerca de 4.

**Notas:**
Linhas sólidas denotam estimativas de $ATT_{es}(d)$ usando Callaway, Goodman-Bacon e Sant’Anna (2024). As áreas sombreadas são intervalos de confiança pontuais de 95%. A curva laranja (azul) define $e_1$ e $e_2$ para 0 e 2 (3 e 4).

**Figura 2. Curvas de Resposta à Dose Estimadas para Efeitos de Curto e Longo Prazo**

Como se deve esperar, as estimativas do estudo de evento na Figura B1 são uma média das estimativas do estudo de evento “alta dose” e “baixa dose” da Figura 1 no texto principal (reproduzidas como Figura B2 para facilitar comparações). Na Figura B1, pode-se ver que as tendências pré-tratamento não paralelas não são uma grande preocupação e que os efeitos de longo prazo são mais fortes que os de curto prazo.

Passamos, em seguida, para as estimativas das curvas de resposta à dose com médias ao longo do tempo, $ATT_{es}(d)$. A Figura 2 no texto principal apresenta os resultados para essas curvas usando os estimadores de Callaway, Goodman-Bacon e Sant’Anna (2024) com splines cúbicos: a curva laranja define $e_1$ e $e_2$ para 0 e 2 (“curto prazo”), e a curva azul usa 3 e 4 (“longo prazo”). Reproduzimos a Figura 2 como Figura B4 abaixo para facilitar comparações. Semelhante ao acima, esperamos que alguns pesquisadores também queiram relatar uma curva de resposta à dose “geral”, por exemplo, definindo $e_1 = 0$ e $e_2 = 4$. Relatamos essas estimativas na Figura B3.

A Figura B3 ecoa as conclusões dos resultados de “longo prazo” da curva de resposta à dose na Figura B4, mostrando que os condados com pontuações de prospectividade mais altas têm maiores efeitos no emprego provenientes do fracking. A Figura B3 também destaca que os efeitos médios de emprego nos primeiros 4 anos após o fracking são igualmente grandes para todos os condados com pontuações acima de cerca de 2.5; não são apenas os condados mais adequados para fracking que impulsionam seus efeitos no mercado de trabalho.

**Notas:**
Linhas sólidas denotam estimativas de $ATT_{es}(d)$ usando Callaway, Goodman-Bacon e Sant’Anna (2024). As áreas sombreadas são intervalos de confiança pontuais de 95%. A curva laranja (azul) define $e_1$ e $e_2$ para 0 e 2 (3 e 4).

**Figura B4. Curvas de Resposta à Dose Estimadas para Efeitos de Curto e Longo Prazo**